在了解完CNN的基本概念之后，本节我们会带领大家用keras来实现一个简单的卷积神经网络，使用的数据是MNIST，MNIST 数据集来自美国国家标准与技术研究所, 数据集由来自 250 个不同人手写的数字构成,包括0-9共10个数字，我们的任务是正确识别出手写数字。本节将会构建一个非常简单并且有代表性的卷及神经网络，预期可达到99%的准确率，读者可通过该例子掌握keras搭建卷及神经网络的要点。

首先载入MNIST数据集，这里直接采用keras内置的获取mnist数据的方法，该方法会把数据下载到对应的目录中，因此执行以下方法时读者需等待片刻。当然读者也可自行下载，地址为：http://yann.lecun.com/exdb/mnist/ 注意，我们这里是一个10分类的问题，读取到的label标签需要做one-hot
处理。

In [21]:
from keras.datasets import mnist
from keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

数据读取完成后，我们对数据的维度进行查看。

In [22]:
X_train.shape

(60000, 28, 28)

可以看到，我们读取的数据是二维的，第一个维度表示的是图片的数量，第二个维度是图片的像素集，我们需要把这一维的数据转换为一个28×28×1的数据集，其中28表示图片的长宽，由于图片是单色道即黑白图片，因此第三个维度是1。我们这里使用reshape方法来改变数据维度。

In [23]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

接下来就是建模的过程，由于模型比较简单，我们这里采用序贯模型Sequential。首先，我们创建第一个卷基层，其中filters表示卷积核的个数，kernel_size表示卷积核的大小，padding表示填充方式，其中包括“valid”与“same”，“valid”代表只进行有效的卷积，即对边界数据不处理，“same”代表保留边界处的卷积结果，通常会导致输出shape与输入shape相同，input_shape表示输入数据的维度，注意，如果是第一层卷基层，必须提供该参数。接下来我们引入一些非线性的变化操作添加一个relu激活函数。最后，使用一个2×2的最大池化层对卷积的输出结果做池化操作，其中pool_size表示赤化层的大小。

In [24]:
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Dense, Conv2D
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5),padding='valid',input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

然后我们定义我们的第二个池化层，这个卷基层和第一层类似，只是卷积核的数量我们改为了64。

In [25]:
model.add(Conv2D(filters=64, kernel_size=(5, 5),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

在经过上面两个卷基层之后，我们的图片大小最后为5×5，由于最后一个卷基层的卷积核是64，所以我们最后拿到的输出结果维度是5×5×64。为了方便加入后面的分类，我们这里加入一个Flatten层，Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。然后连接一个全连接层，并加入relu激活函数，为了防止过拟合，我们添加一个Dropout层，然后我们连接一个10维的全连接层，最后我们把得到的结果输入到softmax层，从而得到最后的概率。

In [26]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

模型建立完成之后，我们来定义损失函数，优化算法与评估指标，其中损失函数我们采用多分类常使用的croos entropy，优化方法我们采用adam，评估指标是准确率。

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

下面就是我们的训练过程了，其中batch_size表示mini-batch的大小，epochs表示训练的迭代次数，verbose表示是否打印日志，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录。

In [30]:
model.fit(X_train, y_train, batch_size=128, epochs=12, verbose=1)

Epoch 1/12
60000/60000 [==============================] - 4s 61us/step - loss: 0.0327 - acc: 0.9904
Epoch 2/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0378 - acc: 0.9891
Epoch 3/12
60000/60000 [==============================] - 4s 59us/step - loss: 0.0313 - acc: 0.9902
Epoch 4/12
60000/60000 [==============================] - 4s 59us/step - loss: 0.0294 - acc: 0.9910
Epoch 5/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0321 - acc: 0.9911
Epoch 6/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0275 - acc: 0.9921
Epoch 7/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0278 - acc: 0.9926
Epoch 8/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0277 - acc: 0.9926
Epoch 9/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0296 - acc: 0.9923
Epoch 10/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.0279 - acc: 0.9929

训练完成后，我们可以在测试集上进行测试，得到最终的测试结果。

In [31]:
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)

print('Test accuracy:', score[1])

Test accuracy: 0.991


可以看到，我们这么简单的一个卷积神经网络，准确率已经高达99.1%，可见卷积层对图像的特征提取是十分有用的，并且依靠卷积核的参数共享，训练效率得到了很大的提升。在后面的文章中，我们会给大家介绍一些更加复杂且效果更好的卷积神经网络，这些网络在各大比赛中都取得了优异的成绩，读者可借鉴这些网络的优点来搭建自己的神经网络，当然也可以采用迁移学习的方法直接使用预先训练好的模型来完成自己的图像识别任务。